# MultiMatest

## Libraries

In [1]:
import parser
import itertools
import pandas as pd

In [2]:
logic = parser.Parser(language=parser.propositional_language())
predicates = logic.language.predicates
operators = logic.language.ops
formula = "a&(a->~b)->a"
formula_parsed = logic.parse(formula)
values = list(range(2))
min_designated = 1
designated_values = list(filter(lambda x: x>=min_designated, values))

In [3]:
def unary_operator(matrix):
    return lambda x: matrix[x]

def binary_operator(matrix):
    return lambda x, y: matrix[x][y]

negation = unary_operator([1,0])

conjuntion = binary_operator([[0,0],
                              [0,1]])

disjunction = binary_operator([[0,1],
                               [1,1]])
                               
implication = binary_operator([[1,1],
                               [0,1]])

logic_functions = {'~':negation, '&':conjuntion, 'v':disjunction, '->': implication}

In [4]:
inputs = formula_parsed.inputs
formula_to_interpret = list(map(lambda x: x.decode('ascii'),formula_parsed.ops.copy()))

In [5]:
formula_to_interpret

['a', 'a', 'b', '~', '->', 'a', '->', '&']

In [6]:
preds = list(sorted(set(filter(lambda x: x in predicates, formula_to_interpret))))

In [7]:
interpretations = list(itertools.product(values, repeat = len(preds)))

In [8]:
interpretations

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [9]:
replacements = [dict(zip(preds, interpretation)) for interpretation in interpretations]

In [10]:
replacements

[{'a': 0, 'b': 0}, {'a': 0, 'b': 1}, {'a': 1, 'b': 0}, {'a': 1, 'b': 1}]

In [11]:
interp = list(map(replacements[0].get, formula_to_interpret, formula_to_interpret))

In [12]:
interp

[0, 0, 0, '~', '->', 0, '->', '&']

In [13]:
inputs

[[], [], [], [-1], [-3, -1], [], [-2, -1], [-7, -1]]

In [14]:
pd.DataFrame([interp, inputs])

,0,1,2,3,4,5,6,7
0,0,0,0,~,->,0,->,&
1,[],[],[],[-1],"[-3, -1]",[],"[-2, -1]","[-7, -1]"


In [15]:
interp_func = list(map(logic_functions.get, interp, interp))

In [16]:
interpret_function = list(map(logic_functions.get, formula_to_interpret, formula_to_interpret))
formula_interpretations = [list(map(replacement.get, interpret_function, interpret_function)) for replacement in replacements]

In [17]:
formula_interpretations[0]

[0,
 0,
 0,
 <function __main__.unary_operator.<locals>.<lambda>(x)>,
 <function __main__.binary_operator.<locals>.<lambda>(x, y)>,
 0,
 <function __main__.binary_operator.<locals>.<lambda>(x, y)>,
 <function __main__.binary_operator.<locals>.<lambda>(x, y)>]

In [18]:
interpretation = formula_interpretations[0]

In [19]:
def evaluate_interpretation(interpretation, inputs, values):
    # The evaluation takes the values from the interpretation
    evaluation = interpretation.copy()

    # Iterates over each of the items of the interpretation
    for index, symbol in enumerate(evaluation):
        # If the symbol is a function, not a value, interpret the expression
        if not symbol in values:
            # The places where the function is applied in the interpretation are always of lower index (the parser orders the expression)
            places = list(map(lambda x: index+x, inputs[index]))
            
            # Picks the values of the places in order to feed the function
            vals = list(map(lambda x: evaluation[x], places))

            # Get the values of the function and replace the item for the value in the evaluation
            val = symbol(*vals)
            evaluation[index] = val

    return evaluation


In [20]:
evaluate_interpretation(formula_interpretations[0],inputs, values)

[0, 0, 0, 1, 1, 0, 0, 0]

In [21]:
evaluations = list(map(lambda interpretation: evaluate_interpretation(interpretation, inputs, values), formula_interpretations))

In [22]:
pd.DataFrame(evaluations, columns=formula_to_interpret,index=interpretations)

,a,a,b,~,->,a,->,&
"(0, 0)",0,0,0,1,1,0,0,0
"(0, 1)",0,0,1,0,1,0,0,0
"(1, 0)",1,1,0,1,1,1,1,1
"(1, 1)",1,1,1,0,0,1,1,1
